In [86]:
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

# Data Preparation

In [83]:
# !kaggle datasets download sleep3r/melanoma <--- download this

In [87]:
import os
import pickle
from typing import Tuple

import cv2
import numpy as np
import pandas as pd

from tqdm import tqdm_notebook

IMG_WIDTH, IMG_HEIGHT = 48, 48
DATA_PATH = './dataset'

In [85]:
def read_image(path: str) -> np.ndarray:
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    return img

def prepare_data(data_path: str = DATA_PATH) -> Tuple[np.ndarray, np.ndarray, np.array, np.array]:
    train_imgs, test_imgs = [], []
    y_train, y_test = [], []

    for dataset in ("train_sep", "valid"):
        for label, label_name in enumerate(("Melanoma", "NotMelanoma")):
            print(dataset, '-', label_name)
            for img in tqdm_notebook(os.listdir(f"{DATA_PATH}/{dataset}/{label_name}")):
                image = read_image(f"{DATA_PATH}/{dataset}/{label_name}/{img}")
                
                if dataset == "train_sep":
                    train_imgs.append(image)
                    y_train.append(label)
                else:
                    test_imgs.append(image)
                    y_test.append(label)
            clear_output(wait=True)
            
    X_train = np.array(train_imgs)
    X_test = np.array(test_imgs)

    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = prepare_data()

# Visualisations

In [88]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [89]:
def plot_samples(X: np.ndarray = X_train, y: np.array = y_train):
    titles = [''] * 10
    titles[2] = "Melanoma"
    titles[-3] = "No melanoma"
    ramdom_img = lambda X: X[np.random.randint(X.shape[0])]
    fig = make_subplots(rows=2, cols=5, start_cell="top-left", subplot_titles=titles)
    
    for col in range(1, 6):
        fig.add_trace(go.Image(z=ramdom_img(X[np.where(y == 0)])), row=1, col=col)
                      
    for col in range(1, 6):
        fig.add_trace(go.Image(z=ramdom_img(X[np.where(y == 1)])), row=2, col=col)
            
    fig.update_layout(title_text="Skin examples")
    fig.show()

In [192]:
def plot_history(history: dict = None, history_path: str = None):
    if history_path:
        with open(history_path, "rb") as f:
            history = pickle.load(f)
            
    fig = make_subplots(rows=1, cols=2, start_cell="top-left", subplot_titles=("Loss", "Acc"))
    
    x = [*range(1, len(history['loss']) + 1)]
    
    fig.add_trace(go.Scatter(x=x, y=history['loss'], name="Train loss"), row=1, col=1)
    fig.add_trace(go.Scatter(x=x, y=history['val_loss'], name="Val loss"), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=x, y=history['accuracy'], name="Train accuracy"), row=1, col=2)
    fig.add_trace(go.Scatter(x=x, y=history['val_accuracy'], name="Val accuracy"), row=1, col=2)
    
    fig.update_layout(title_text="Validation curves")
    fig.show()

In [181]:
plot_samples()

# Evaluation

In [155]:
import plotly.figure_factory as ff
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

In [219]:
def evaluate(model, X_test, y_test):
    if not isinstance(model, Model):
        preds = model.predict_classes(X_test, verbose=1)
    else:
        preds = np.argmax(model.predict(X_test, verbose=1), axis=1)

    m = confusion_matrix(y_test, preds)

    fig = ff.create_annotated_heatmap(m, x=['Positive', 'Negative'], y=['Negative', 'Positive'], 
                                      colorscale=[[0, 'navy'], [1, 'plum']])
    fig.update_layout(
        title="Confusion matrix",
        xaxis_title="Predicted",
        yaxis_title="Actual",
    )
    fig.show()

    fpr, tpr, threshold = roc_curve(y_test, preds)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], line={"dash": 'dot'}, name='Random'))
    fig.add_trace(go.Scatter(x=fpr, y=tpr, name='Fitted model'))
    fig.update_layout(title=f"ROC-AUC: {round(roc_auc_score(y_test, preds), 3)}")
    fig.show()

# Keras nets

In [227]:
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten, Conv2D, \
                         MaxPooling2D, Dropout, BatchNormalization, MaxPool2D, Activation, Add, ZeroPadding2D
from keras.regularizers import l2
from keras.models import Model, Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.utils.np_utils import to_categorical

### normal CNN

In [81]:
def create_model_cnn(input_shape: tuple, n_classes: int = 2, weights_path: str = None):
    model = Sequential()
    
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu", input_shape=input_shape))
    model.add(Conv2D(filters=128, kernel_size=(5, 5), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(1024, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dense(n_classes, activation="softmax"))
    
    if weights_path:
        model.load_weights(weights_path)

    model.compile(loss="categorical_crossentropy", 
                  optimizer="adam", 
                  metrics=["accuracy"])
    
    return model

In [82]:
model = create_model_cnn(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), weights_path="./models/CNN/model.h5")

In [196]:
plot_history(history_path="./history/CNN.history")
evaluate(model, X_test, y_test)

3562/3562 [==============================] - 21s 6ms/step


### deep CNN

In [200]:
def create_model_deep_cnn(input_shape: tuple, n_classes: int = 2, weights_path: str = None):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', input_shape=input_shape))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPool2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(n_classes, activation='softmax'))
    
    if weights_path:
        model.load_weights(weights_path)

    model.compile(optimizer=tf.train.AdamOptimizer(), 
                  loss=tf.losses.softmax_cross_entropy,
                  metrics=['accuracy'])
   
    return model

In [205]:
model = create_model_deep_cnn(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), weights_path="./models/deepCNN/model.h5")

Instructions for updating:
Use tf.cast instead.


In [206]:
plot_history(history_path="./history/deepCNN.history")
evaluate(model, X_test, y_test)

3562/3562 [==============================] - 11s 3ms/step


### ResNet

In [207]:
def resnet_block(x, size, kernel_size=(5, 5)):
    y = BatchNormalization(momentum=0.9)(x)
    y = Activation('relu')(y)
    y = Conv2D(size, kernel_size=kernel_size, padding='same')(y)
    y = BatchNormalization(momentum=0.9)(y)
    y = Activation('relu')(y)
    y = Conv2D(size, kernel_size=kernel_size, padding='same')(y)
    return Add()([x, y])

def create_model_resnet(input_shape: tuple, n_classes: int = 2, weights_path: str = None, heads_number=1):
    inputs = Input(shape=input_shape)
    
    layer = Conv2D(32, kernel_size=(5, 5), strides=(2, 2), activation='relu')(inputs)
    layer = resnet_block(layer, 32)
    layer = resnet_block(layer, 32)
    
    layer = Conv2D(64, kernel_size=(2, 2), strides=(2, 2), activation='relu')(layer)
    layer = resnet_block(layer, 64)
    layer = resnet_block(layer, 64)
    
    layer = Conv2D(128, kernel_size=(2, 2), strides=(2, 2), activation='relu')(layer)
    layer = resnet_block(layer, 128)
    layer = resnet_block(layer, 128)
    
    layer = Conv2D(256, kernel_size=(2, 2), strides=(2, 2), activation='relu')(layer)
    layer = resnet_block(layer, 256)
    layer = resnet_block(layer, 256)
    layer = BatchNormalization(momentum=0.9)(layer)
    
    layer = Flatten()(layer)
    
    layer = Dense(2048, activation='relu')(layer)
    outputs = [Dense(n_classes)(layer) for _ in range(heads_number)]
    
    model = Model(inputs=inputs, outputs=outputs)
    
    if weights_path:
        model.load_weights(weights_path)
        
    model.compile(optimizer=tf.train.AdamOptimizer(), 
                  loss=[tf.losses.softmax_cross_entropy for _ in range(heads_number)], 
                  metrics=['accuracy'])
    
    return model

In [212]:
model = create_model_resnet(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), weights_path="./models/resNet/model.h5")

In [220]:
plot_history(history_path="./history/resNet.history")
evaluate(model, X_test, y_test)

3562/3562 [==============================] - 15s 4ms/step


### AlexNet

In [228]:
def create_model_alexnet(input_shape: tuple, n_classes: int = 2, l2_reg: float = 0.0, weights_path: str = None):
    model = Sequential()

    model.add(Conv2D(32, (5, 5), input_shape=input_shape, padding='same', kernel_regularizer=l2(l2_reg)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (5, 5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(2048))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(n_classes))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    if weights_path:
        model.load_weights(weights_path)
    
    model.compile(optimizer=tf.train.AdamOptimizer(), 
                  loss=tf.losses.softmax_cross_entropy,
                  metrics=['accuracy'])

    return model

In [229]:
model = create_model_alexnet(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), weights_path="./models/alexNet/model.h5")

In [230]:
plot_history(history_path="./history/alexNet.history")
evaluate(model, X_test, y_test)

3562/3562 [==============================] - 22s 6ms/step


## VGG16 Pretrained

In [73]:
def create_model_vgg(input_shape: tuple, weights_path: str = None):
    vgg = VGG16(input_shape=input_shape, weights='imagenet', include_top=False)

    for layer in vgg.layers:
        layer.trainable = False

    x = vgg.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dense(1024, activation="relu")(x)
    predictions = Dense(2, activation="softmax")(x)

    model = Model(inputs = vgg.input, outputs = predictions)
        
    if weights_path:
        model.load_weights(weights_path)
        
    model.compile(loss="categorical_crossentropy", 
                      optimizer="adam", 
                      metrics=["accuracy"])
    
    return model

In [232]:
model = create_model_vgg(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), weights_path="./models/vggPretrained/model.h5")

In [233]:
plot_history(history_path="./history/vggPretrained.history")
evaluate(model, X_test, y_test)

3562/3562 [==============================] - 44s 12ms/step


# Train

In [15]:
def shuffle_data(X: np.ndarray, y: np.array) -> Tuple[np.ndarray, np.array]:
    assert X.shape[0] == y.shape[0]
    
    shuffled_indices = np.arange(X.shape[0])
    np.random.shuffle(shuffled_indices)
    X, y = X[shuffled_indices], y[shuffled_indices]
    return X, y

In [16]:
def prepare_data(*data: tuple) -> tuple:
    X_train, X_test, y_train, y_test = data
    
    X_train_norm = X_train / 255.0
    X_test_norm = X_test / 255.0

    y_train_onehot = to_categorical(y_train, 2)
    y_test_onehot = to_categorical(y_test, 2)
    
    X_train_norm, y_train_onehot = shuffle_data(X_train_norm, y_train_onehot)
    X_test_norm, y_test_onehot = shuffle_data(X_test_norm, y_test_onehot)
    
    return X_train_norm, X_test_norm, y_train_onehot, y_test_onehot

In [17]:
def train_net(model, *data: tuple, name: str, epochs = 40, batch_size=100):
    X_train, X_test, y_train, y_test = data
    
    history = model.fit(X_train_norm, y_train_onehot,
                        validation_data=(X_test_norm, y_test_onehot),
                        batch_size=batch_size, epochs=epochs)

    model.save(f"./models/{name}/model.h5")
    with open(f"./history/{name}.history", "wb") as f:
        pickle.dump(history.history, f)

In [20]:
X_train_norm, X_test_norm, y_train_onehot, y_test_onehot = prepare_data(X_train, X_test, y_train, y_test)
data = X_train_norm, X_test_norm, y_train_onehot, y_test_onehot

In [178]:
model = create_model_cnn(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), n_classes=2)
train_net(model, *data, name="CNN", epochs=10, batch_size=50)

Train on 10682 samples, validate on 3562 samples
Epoch 1/10
10682/10682 [==============================] - 248s 23ms/step - loss: 0.5195 - accuracy: 0.7731 - val_loss: 3.4484 - val_accuracy: 0.5000
Epoch 2/10
10682/10682 [==============================] - 251s 23ms/step - loss: 0.3965 - accuracy: 0.8314 - val_loss: 5.6965 - val_accuracy: 0.5000
Epoch 3/10
10682/10682 [==============================] - 249s 23ms/step - loss: 0.3678 - accuracy: 0.8474 - val_loss: 0.8061 - val_accuracy: 0.6951
Epoch 4/10
10682/10682 [==============================] - 258s 24ms/step - loss: 0.3215 - accuracy: 0.8782 - val_loss: 0.5029 - val_accuracy: 0.7426
Epoch 5/10
10682/10682 [==============================] - 254s 24ms/step - loss: 0.2693 - accuracy: 0.9081 - val_loss: 4.8292 - val_accuracy: 0.5000
Epoch 6/10
10682/10682 [==============================] - 254s 24ms/step - loss: 0.2423 - accuracy: 0.9163 - val_loss: 1.3603 - val_accuracy: 0.5649
Epoch 7/10
10682/10682 [==============================] -

In [179]:
model = create_model_deep_cnn(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), n_classes=2)
train_net(model, *data, name="deepCNN", epochs=10, batch_size=50)

Train on 10682 samples, validate on 3562 samples
Epoch 1/10
10682/10682 [==============================] - 83s 8ms/step - loss: 0.5226 - accuracy: 0.7768 - val_loss: 0.7704 - val_accuracy: 0.5390
Epoch 2/10
10682/10682 [==============================] - 84s 8ms/step - loss: 0.4826 - accuracy: 0.8209 - val_loss: 0.5033 - val_accuracy: 0.7979
Epoch 3/10
10682/10682 [==============================] - 84s 8ms/step - loss: 0.4763 - accuracy: 0.8295 - val_loss: 0.6468 - val_accuracy: 0.6491
Epoch 4/10
10682/10682 [==============================] - 82s 8ms/step - loss: 0.4614 - accuracy: 0.8468 - val_loss: 0.8016 - val_accuracy: 0.5093
Epoch 5/10
10682/10682 [==============================] - 116s 11ms/step - loss: 0.4602 - accuracy: 0.8474 - val_loss: 0.4837 - val_accuracy: 0.8234
Epoch 6/10
10682/10682 [==============================] - 90s 8ms/step - loss: 0.4497 - accuracy: 0.8581 - val_loss: 0.4402 - val_accuracy: 0.8697
Epoch 7/10
10682/10682 [==============================] - 80s 8ms/s

In [180]:
model = create_model_resnet(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), n_classes=2)
train_net(model, *data, name="resNet", epochs=10, batch_size=50)

Train on 10682 samples, validate on 3562 samples
Epoch 1/10
10682/10682 [==============================] - 235s 22ms/step - loss: 0.6377 - accuracy: 0.7543 - val_loss: 0.7872 - val_accuracy: 0.5497
Epoch 2/10
10682/10682 [==============================] - 201s 19ms/step - loss: 0.4097 - accuracy: 0.8235 - val_loss: 0.3946 - val_accuracy: 0.8377
Epoch 3/10
10682/10682 [==============================] - 309s 29ms/step - loss: 0.3759 - accuracy: 0.8377 - val_loss: 0.4857 - val_accuracy: 0.7622
Epoch 4/10
10682/10682 [==============================] - 328s 31ms/step - loss: 0.3614 - accuracy: 0.8460 - val_loss: 0.4283 - val_accuracy: 0.8467
Epoch 5/10
10682/10682 [==============================] - 326s 31ms/step - loss: 0.3520 - accuracy: 0.8488 - val_loss: 1.3282 - val_accuracy: 0.6463
Epoch 6/10
10682/10682 [==============================] - 329s 31ms/step - loss: 0.3308 - accuracy: 0.8581 - val_loss: 0.4461 - val_accuracy: 0.8220
Epoch 7/10
10682/10682 [==============================] -

In [183]:
model = create_model_alexnet(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), n_classes=2, l2_reg=0.5)
train_net(model, *data, name="alexNet", epochs=10, batch_size=50)

Train on 10682 samples, validate on 3562 samples
Epoch 1/10
10682/10682 [==============================] - 236s 22ms/step - loss: 0.8450 - accuracy: 0.8124 - val_loss: 0.8185 - val_accuracy: 0.5000
Epoch 2/10
10682/10682 [==============================] - 242s 23ms/step - loss: 0.4940 - accuracy: 0.8403 - val_loss: 0.8033 - val_accuracy: 0.5000
Epoch 3/10
10682/10682 [==============================] - 242s 23ms/step - loss: 0.4784 - accuracy: 0.8540 - val_loss: 0.4998 - val_accuracy: 0.8355
Epoch 4/10
10682/10682 [==============================] - 256s 24ms/step - loss: 0.4709 - accuracy: 0.8605 - val_loss: 0.5237 - val_accuracy: 0.8113
Epoch 5/10
10682/10682 [==============================] - 249s 23ms/step - loss: 0.4617 - accuracy: 0.8651 - val_loss: 0.6634 - val_accuracy: 0.6221
Epoch 6/10
10682/10682 [==============================] - 249s 23ms/step - loss: 0.4574 - accuracy: 0.8710 - val_loss: 0.7282 - val_accuracy: 0.5404
Epoch 7/10
10682/10682 [==============================] -

In [75]:
model = create_model_vgg(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
train_net(model, *data, name="vggPretrained", epochs=10, batch_size=40)

Train on 10682 samples, validate on 3562 samples
Epoch 1/10
10682/10682 [==============================] - 191s 18ms/step - loss: 0.4916 - accuracy: 0.7932 - val_loss: 0.4330 - val_accuracy: 0.8071
Epoch 2/10
10682/10682 [==============================] - 192s 18ms/step - loss: 0.3625 - accuracy: 0.8399 - val_loss: 0.3633 - val_accuracy: 0.8285
Epoch 3/10
10682/10682 [==============================] - 219s 20ms/step - loss: 0.3232 - accuracy: 0.8593 - val_loss: 0.3500 - val_accuracy: 0.8549
Epoch 4/10
10682/10682 [==============================] - 211s 20ms/step - loss: 0.3163 - accuracy: 0.8599 - val_loss: 0.3881 - val_accuracy: 0.8296
Epoch 5/10
10682/10682 [==============================] - 201s 19ms/step - loss: 0.2991 - accuracy: 0.8659 - val_loss: 0.3655 - val_accuracy: 0.8501
Epoch 6/10
10682/10682 [==============================] - 187s 18ms/step - loss: 0.2937 - accuracy: 0.8749 - val_loss: 0.3672 - val_accuracy: 0.8470
Epoch 7/10
10682/10682 [==============================] -